IMPORTS and GLOBALS

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm.notebook import tqdm
from livelossplot import PlotLosses 
import numpy as np
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM

# model_path: str = "/home/sajib/Documents/llm-attacks/llama/llama-2-7b-chat-hf"
# use Samuel's directory instead
model_path: str = "/home/samuel/research/llmattacks/llm-attacks/DIR/llama-2/llama/Llama-2-7b-chat-hf"

device: str = "cuda:0"
num_steps: int = 100
num_tokens: int = 100
step_size: float = 0.1 # A smaller learning_rate does NOT work!
# generate_interval: int = 500
seed: int = 42
load_dataset = True
# verbose = True
# early_stopping = True

In [2]:
def load_model_and_tokenizer(model_path, tokenizer_path=None, device="cuda:0", **kwargs):
    # from llm-attacks
    model = (
        AutoModelForCausalLM.from_pretrained(
            model_path, torch_dtype=torch.float16, trust_remote_code=True, **kwargs
        ).to(device).eval()
    )

    tokenizer_path = model_path if tokenizer_path is None else tokenizer_path

    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_path, trust_remote_code=True, use_fast=False
    )

    if "llama-2" in tokenizer_path:
        tokenizer.pad_token = tokenizer.unk_token
        tokenizer.padding_side = "left"
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

def get_embedding_matrix(model):
    # from llm-attacks
    if isinstance(model, LlamaForCausalLM):
        return model.model.embed_tokens.weight
    else:
        raise ValueError(f"Unknown model type: {type(model)}")

In [ ]:
if seed is not None:
        torch.manual_seed(seed)

model, tokenizer = load_model_and_tokenizer(
        model_path, low_cpu_mem_usage=True, use_cache=False, device=device
    )


In [4]:
def get_tokens(input_string):
    return torch.tensor(tokenizer(input_string)["input_ids"], device=device)


def create_one_hot_and_embeddings(tokens, embed_weights):
    one_hot = torch.zeros(
        tokens.shape[0], embed_weights.shape[0], device=device, dtype=embed_weights.dtype
    )
    one_hot.scatter_(
        1,
        tokens.unsqueeze(1),
        torch.ones(one_hot.shape[0], 1, device=device, dtype=embed_weights.dtype),
    )
    embeddings = (one_hot @ embed_weights).unsqueeze(0).data
    return one_hot, embeddings

In [5]:
# def calc_loss(model, embeddings_user, embeddings_adv, embeddings_target, targets):
#     full_embeddings = torch.hstack([embeddings_user, embeddings_adv, embeddings_target])
#     logits = model(inputs_embeds=full_embeddings).logits
#     loss_slice_start = len(embeddings_user[0]) + len(embeddings_adv[0])
#     loss = nn.CrossEntropyLoss()(logits[0, loss_slice_start - 1 : -1, :], targets)
#     return loss, logits[:, loss_slice_start-1:, :]
#     # return loss, logits[:, loss_slice_start:, :]

In [6]:
# A safer version of the calc_loss function.
def calc_loss(model, embeddings_user, embeddings_adv, embeddings_target, targets):
    full_embeddings = torch.hstack([embeddings_user, embeddings_adv, embeddings_target])
    
    # Check for NaN or extreme values in the embeddings
    if torch.isnan(full_embeddings).any():
        raise ValueError("NaN detected in input embeddings")
    if torch.max(full_embeddings).item() > 1e6 or torch.min(full_embeddings).item() < -1e6:
        print("Warning: Extreme values detected in embeddings")

    # Pass through model and get logits
    logits = model(inputs_embeds=full_embeddings).logits

    # Check for NaN or extreme values in logits
    if torch.isnan(logits).any():
        raise ValueError("NaN detected in logits")
    if torch.max(logits).item() > 1e6 or torch.min(logits).item() < -1e6:
        print("Warning: Extreme values detected in logits")
    
    # Apply optional clipping to logits
    logits = torch.clamp(logits, min=-1e4, max=1e4)

    # Compute the cross-entropy loss
    loss_slice_start = len(embeddings_user[0]) + len(embeddings_adv[0])
    loss = nn.CrossEntropyLoss()(logits[0, loss_slice_start - 1 : -1, :], targets)

    return loss, logits[:, loss_slice_start - 1:, :]


BEGIN ATTACK HERE

In [7]:
# def simplex_projection(s):
#     """
#     Project the input tensor s onto the probability simplex.

#     Parameters:
#     - s: Updated token (PyTorch tensor)

#     Returns:
#     - p: Projected tensor (PyTorch tensor)
#     """
#     if s.numel() == 0:
#         raise ValueError("Input tensor s must not be empty")
    
#     # Step 1: Sort s into mu in descending order
#     mu, _ = torch.sort(s, descending=True)
    
#     # Step 2: Compute rho
#     cumulative_sum = torch.cumsum(mu, dim=0)
#     arange = torch.arange(1, s.size(0) + 1, device=s.device)
#     condition = mu - (cumulative_sum - 1) / arange > 0
#     # Handle numerical precision issues by adding a small epsilon
#     if not condition.any():
#         epsilon = 1e-10
#         condition = (mu + epsilon) - (cumulative_sum - 1) / arange > 0

#     # Step 2: Compute rho
#     # cumulative_sum = torch.cumsum(mu, dim=0)
#     # arange = torch.arange(1, s.size(0) + 1, device=s.device)
#     # condition = mu - (cumulative_sum - 1) / arange > 0
#     nonzero_indices = torch.nonzero(condition, as_tuple=False)
#     if nonzero_indices.size(0) == 0:
#         rho = 1.0  # Default rho to 1 if no valid condition is found
#     else:
#         rho = nonzero_indices[-1].item() + 1.0
    
#     # Step 3: Compute psi
#     psi = (cumulative_sum[rho - 1] - 1) / rho
    
#     # Step 4: Compute p
#     p = torch.clamp(s - psi, min=0)
    
#     return p

def simplex_projection(s, epsilon=1e-10):
    if s.numel() == 0:
        raise ValueError("Input tensor s must not be empty")
    
    # Step 1: Sort s into mu in descending order
    mu, _ = torch.sort(s, descending=True)
    
    # Step 2: Compute rho
    cumulative_sum = torch.cumsum(mu, dim=0)
    arange = torch.arange(1, s.size(0) + 1, device=s.device)
    condition = mu - (cumulative_sum - 1 + epsilon) / (arange + epsilon) > 0

    nonzero_indices = torch.nonzero(condition, as_tuple=False)
    if nonzero_indices.size(0) == 0:
        rho = 1
    else:
        rho = nonzero_indices[-1].item() + 1

    # Step 3: Compute psi
    psi = (cumulative_sum[rho - 1] - 1) / rho
    
    # Step 4: Compute p
    p = torch.clamp(s - psi, min=0)
    
    return p


def project_rows_to_simplex(matrix):
    """
    Apply the simplex projection row-wise to a 2D tensor.

    Parameters:
    - matrix: 2D tensor (PyTorch tensor)

    Returns:
    - projected_matrix: Row-wise simplex projected 2D tensor (PyTorch tensor)
    """
    projected_matrix = torch.zeros_like(matrix)
    for i in range(matrix.size(0)):
        projected_matrix[i] = simplex_projection(matrix[i])
    return projected_matrix


In [8]:
# def entropy_projection(s, target_entropy=2):
#     """
#     Project the input tensor s onto the entropy-constrained space.

#     Parameters:
#     - s: Relaxed token (PyTorch tensor with values in [0, 1])
#     - Sq: Target entropy (default is 2)
#     - epsilon: Small value to avoid division by zero or log(0)

#     Returns:
#     - p: Projected tensor (PyTorch tensor)
#     """
#     # Step 1: Compute center c
#     mask = (s > 0).float()  # Indicator function I[s > 0]
#     non_zero_count = torch.sum(mask)
#     c = mask / non_zero_count  # Center c

#     # Step 2: Compute radius R
#     gini_index = 1 - torch.square(s).sum() # PGD uses 's', NOT 'mask'
#     R = torch.sqrt(1.0 - gini_index - (1.0/non_zero_count))
#     # Compute Euclidean norm of (s - c)
#     norm_s_c = torch.norm(s - c)

#     # Step 4: Check if R >= ||s - c||
#     if R >= norm_s_c:
#         # Step 5: Return s if the condition is satisfied
#         return s
#     else:
#         # Step 7: Project to simplex with scaling
#         scaled_s = R / norm_s_c * (s - c) + c
#         return simplex_projection(scaled_s)


def entropy_projection(s, target_entropy=2, epsilon=1e-10):
    mask = (s > epsilon).float()
    non_zero_count = torch.sum(mask) + epsilon  # Prevent division by zero
    c = mask / non_zero_count

    # Step 2: Compute radius R
    gini_index = 1 - torch.square(mask).sum()  # Ensure gini_index >= 0 ; PGD uses 's', NOT 'mask'
    gini_index = torch.clamp(gini_index, min=0, max=1)  # Keep it in valid range
    R = torch.sqrt(1.0 - gini_index + epsilon - (1.0 / non_zero_count))

    # Compute Euclidean norm of (s - c)
    norm_s_c = torch.norm(s - c)

    # Check if R >= ||s - c||
    if R >= norm_s_c:
        return s
    else:
        scaled_s = R / (norm_s_c + epsilon) * (s - c) + c
        return simplex_projection(scaled_s)


def project_rows_to_entropy(matrix):
    """
    Apply the simplex projection row-wise to a 2D tensor.

    Parameters:
    - matrix: 2D tensor (PyTorch tensor)

    Returns:
    - projected_matrix: Row-wise simplex projected 2D tensor (PyTorch tensor)
    """
    projected_matrix = torch.zeros_like(matrix)
    for i in range(matrix.size(0)):
        projected_matrix[i] = entropy_projection(matrix[i])
    return projected_matrix


BEGIN ATTACK HERE

In [ ]:
import csv
embed_weights = get_embedding_matrix(model)
dataset_path: str = "/home/samuel/research/llmattacks/llm-attacks/data/advbench/harmful_behaviors.csv"
if load_dataset:
    reader = csv.reader(open(dataset_path, 'r'))
    next(reader)
else:
    print("Error!")
harmful_behaviors = list(reader)[0:1]
print(len(harmful_behaviors))
for each_behavior in harmful_behaviors:
    print(each_behavior)
    # user_prompt, target = each_behavior
    # print('Query:', user_prompt)
    # user_prompt_tokens = get_tokens(user_prompt)    
    # _, embeddings_user = create_one_hot_and_embeddings(user_prompt_tokens, embed_weights)
    # generated_output =  model.generate(inputs_embeds=embeddings_user, max_length=len(user_prompt_tokens)).squeeze()
    # generated_output_string = tokenizer.decode(generated_output, skip_special_tokens=True)
    # print('Response:', generated_output_string,end='\n\n\n')
    # each_behavior.append(generated_output_string)

In [ ]:
# Use Adam from optim
# Update the one_hot encodings using Gradient Descent.
# Use random initialization for Adversarial One Hot
from math import * 
import json
import math
import pandas as pd
import torch.nn.functional as F
embed_weights = get_embedding_matrix(model)
# reader = [[user_prompt, target]]
optimization_results = []
# Create the following directories if they do NOT exist
directories = ["./CSV_Files", "./JSON_Files", "./TORCH_Files"]
for directory in directories:
    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists.")
# JSON file to dump the model generated outputs 
json_file_path = "./JSON_Files/output_PGD_w_Adam_Cosine("+str(len(harmful_behaviors))+"_behaviors)("+str(num_steps)+"_steps).json"
# json_file_path = "./JSON_Files/output_ProGD_with_Cosine(random_init)_"+str(len(harmful_behaviors))+"_behaviors("+str(num_steps)+"Iterations).json"

# for row in reader:
for row in tqdm(harmful_behaviors, desc="Optimizing prompts"):
    iteration_result = {}
    user_prompt, target = row
    print(user_prompt)
    # adv_string_tokens = get_tokens(adv_string_init)[1:]
    user_prompt_tokens = get_tokens(user_prompt)
    # target = unsafe_target    
    target_tokens = get_tokens(target)[1:]
    plotlosses = PlotLosses()
    plotlosses = PlotLosses(groups={'loss': ['continuous_loss', 'discrete_loss']})
    # Plotloss to graph the loss
    # Create both embeddings and one_hot for Safe and Unsafe target Seperate (To plot the losses)
    # unsafe_target_tokens = get_tokens(unsafe_target)[1:]    
    # safe_target_tokens = get_tokens(safe_target)[1:]
    # adv_suffix_tokens = get_tokens(adv_suffix)[1:]
    # one_hot_unsafe_target, embeddings_unsafe_target = create_one_hot_and_embeddings(unsafe_target_tokens, embed_weights)
    # one_hot_safe_target, embeddings_safe_target = create_one_hot_and_embeddings(safe_target_tokens, embed_weights)
    one_hot_inputs, embeddings_user = create_one_hot_and_embeddings(user_prompt_tokens, embed_weights)
    # one_hot_adv, _ = create_one_hot_and_embeddings(adv_string_tokens, embed_weights)
    one_hot_adv = F.softmax(torch.rand(20, 32000, dtype=torch.float16).to(device=device), dim=1)
    one_hot_target, embeddings_target = create_one_hot_and_embeddings(target_tokens, embed_weights)
    best_disc_loss = np.inf
    # cur_loss_list = []
    effective_adv_one_hot = one_hot_adv.detach()
    effective_adv_embedding = (one_hot_adv @ embed_weights).unsqueeze(0)
    one_hot_adv.requires_grad_()
    # Initialize Adam optimizer with user-defined epsilon value
    optimizer = optim.Adam([one_hot_adv], lr=step_size, eps=1e-4)
    # Initialize lr scheduler (Cosine Annealing with Warm Restarts)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0, last_epoch=-1)
    scheduler_cycle = 0
    # Specify the filename
    last_three_words = lambda text: '_'.join(text.split()[-3:])
    first_three_words = lambda text: '_'.join(text.split()[0:3])
    csv_filename = './CSV_Files/output_PGD_w_Adam_Cosine('+first_three_words(user_prompt)+".."+last_three_words(user_prompt)+')('+str(num_steps)+'_steps).csv'
    # csv_filename = './CSV_Files/output_ProGD_with_Cosine(random_init)_'+last_three_words(user_prompt)+'('+str(num_steps)+'_iterations).csv'
    # Generate column names
    column_names = ['epoch', 'cycle', 'learning_rate', 'continuous_loss', 'discrete_loss']
                    # "unsafe_loss", "safe_loss"] 
                    
    # Adding 'max_1' to 'max_20' column names using a loop
    for i in range(1, 21):
        column_names.append(f'max_{i}')
    for i in range(1, 21):
        column_names.append(f'token_id_{i}')

    # Create an empty DataFrame
    df = pd.DataFrame(columns=column_names)
    # PHASE_2: bool=False
    BREAK_IT: bool=False

    for epoch_no in tqdm(range(num_steps)):
        optimizer.zero_grad()
        # model.zero_grad()
        embeddings_adv = (one_hot_adv @ embed_weights).unsqueeze(0)
        cross_entropy_loss, logits = calc_loss(model, embeddings_user, embeddings_adv, embeddings_target, one_hot_target)
        continuous_loss = cross_entropy_loss.detach().cpu().item()
        # Break the loop if loss becomes "NaN"
        if math.isnan(continuous_loss):
            print("The value is NaN. Skipping behavior.")
            BREAK_IT=True # Go to the Next Behavior

        # # Discretization part
        max_values = torch.max(one_hot_adv, dim=1)
        adv_token_ids = max_values.indices
        one_hot_discrete = torch.zeros(
            adv_token_ids.shape[0], embed_weights.shape[0], device=device, dtype=embed_weights.dtype
        )
        one_hot_discrete.scatter_(
            1,
            adv_token_ids.unsqueeze(1),
            torch.ones(one_hot_adv.shape[0], 1, device=device, dtype=embed_weights.dtype),
        )
        # Use one_hot_discrete to print Tokens
        # What other techniques Can we use here to discretize the one_hot encodings?
        # print("Adversarial tokens: ", tokenizer.decode(one_hot_discrete.argmax(dim=1)) )
        # Use discrete tokens to calculate loss
        embeddings_adv_discrete = (one_hot_discrete @ embed_weights).unsqueeze(0)
        disc_loss, _ = calc_loss(model, embeddings_user, embeddings_adv_discrete, embeddings_target, one_hot_target)
        # If loss improves, save it as x_best
        discrete_loss =  disc_loss.detach().cpu().item()
        # cur_loss_list.append(cur_loss)
        if discrete_loss < best_disc_loss:
            # print(f"########## {discrete_loss} #########")
            best_disc_loss = discrete_loss
            best_loss_at_epoch = epoch_no
            effective_adv_embeddings = embeddings_adv_discrete
            effective_adv_one_hot = one_hot_discrete
        else :
            pass
        # # Update plotlosses with both discrete and continuous loss
        # plotlosses.update({
        #     "continuous_loss": continuous_loss,
        #     "discrete_loss": discrete_loss
        # })
        # plotlosses.send()
        cross_entropy_loss.backward()
        # Get the scheduler's state to get learning_rate
        scheduler_state = scheduler.state_dict()
        # plotlosses.update({"loss": loss.detach().cpu().numpy()})
        # grad = one_hot_adv.grad.clone()
        torch.nn.utils.clip_grad_norm_([one_hot_adv], max_norm=1.0)
        # update parameters
        optimizer.step()
        # update scheduler
        scheduler.step()
        # Get the current cycle number somehow
        model.zero_grad()
        # optimizer.param_groups[0]['lr'] = adjust_learning_rate(lr=step_size, iteration=i)

        # Apply Simplex & Entropy
        simplices = project_rows_to_simplex(one_hot_adv)
        regularized_simplices = project_rows_to_entropy(simplices)
        one_hot_adv.data = regularized_simplices.data 
        
        one_hot_adv.grad.zero_()
        # Dump Ouput values to a CSV file
        # Convert max_values to a NumPy array
        max_values_array = max_values.values.detach().cpu().numpy()
        token_ids_array = adv_token_ids.detach().cpu().numpy()
        # Get the scheduler's state and learning_rate
        scheduler_lr = scheduler_state['_last_lr'][0]
        # Compute Cycle
        if scheduler_state['T_cur'] == 0:
                scheduler_cycle += 1
        # Create the Initial array       
        prepend_array = np.array([epoch_no, scheduler_cycle, scheduler_lr,
                                    continuous_loss, discrete_loss]) 
                                #   unsafe_loss, safe_loss])  
        
        
        # Concatenate the arrays
        row = np.concatenate((prepend_array, max_values_array, token_ids_array))
        new_row = pd.Series(row, index=df.columns)
        df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
        # df = df.append(pd.Series(row, index=df.columns), ignore_index=True)
        # # Save log data to CSV file periodically
        # if epoch_no % 10 == 0:
        #     df.to_csv(csv_filename, index=False)
        # Something went wrong; so break it and go to the next behavior.
        if BREAK_IT==True:
            break
    # End of optimizations            
    # Write to CSV file
    # df.to_csv(csv_filename, index=False)
    # Token and Output Generation Step
    inputs_token_ids = one_hot_inputs.argmax(dim=1)
    adv_token_ids = effective_adv_one_hot.argmax(dim=1)
    adv_suffix_list = adv_token_ids.tolist()
    final_string_ids = torch.hstack([inputs_token_ids, adv_token_ids])
    outputs = []
    generation_loop = range(0)
    for loop in tqdm(generation_loop, desc="Generating outputs", leave=False):
        # generated_output = model.generate(inputs_embeds=final_prompt_embeds, max_length=num_tokens).squeeze()
        # generated_output =  model.generate(inputs_embeds=final_prompt_embeds, max_length=num_tokens, do_sample=False).squeeze()
        # generated_output_string = tokenizer.decode(generated_output, skip_special_tokens=True)
        generated_output = model.generate(final_string_ids.unsqueeze(0), max_length=num_tokens, pad_token_id=tokenizer.pad_token_id)
        generated_output_string = tokenizer.decode(generated_output[0][1:].cpu().numpy(), skip_special_tokens=True).strip()
        outputs.append(generated_output_string)

    iteration_result = {
        "harmful-behaviour": user_prompt,
        "suffix_token_ids": adv_suffix_list,
        "target": target,
        "epoch_no": epoch_no,
        "continuous_loss": continuous_loss,
        "discrete_loss": discrete_loss,
        "best_disc_loss":best_disc_loss,
        "epch_at_best_disc_loss": best_loss_at_epoch,
        "outputs": outputs
    }
    optimization_results.append(iteration_result)
    # Update the JSON File
    with open(json_file_path, "w") as f:
        json.dump(optimization_results, f, indent=4)

## Additional Codes

In [11]:
# # File paths to the .pt files
# # tensor_filenames = ['Safe_one_hot_adv.pt', 'Unsafe_one_hot_adv.pt']
# for row in harmful_behaviors:
#     iteration_result = {}
#     user_prompt, _ = row
#     print(f'\n\n{user_prompt}')
#     print('________________________________________________________________________________')
#     tensor_file_unsafe = './TORCH_Files/ProGD_with_Cosine_Unsafe_1_hot_adv_'+last_three_words(user_prompt)+'_'+first_three_words(unsafe_target)+'.pt'
#     tensor_file_safe = './TORCH_Files/ProGD_with_Cosine_Safe_1_hot_adv_'+last_three_words(user_prompt)+'_'+first_three_words(safe_target)+'.pt'
#     tensor_filenames = [tensor_file_unsafe, tensor_file_safe]
#     # Load the tensors from the .pt files
#     loaded_tensors = [torch.load(filename) for filename in tensor_filenames]

#     # Now, loaded_tensors is a list containing the loaded tensors
#     tensor1 = loaded_tensors[0]
#     tensor2 = loaded_tensors[1]

#     # Move tensors to CPU and convert to float32 if necessary
#     tensor1_cpu = tensor1.cpu().float()
#     tensor2_cpu = tensor2.cpu().float()

#     # Example usage: Compute cosine similarity and L2 distance
#     import torch.nn.functional as F

#     # Compute cosine similarity
#     cosine_similarity = F.cosine_similarity(tensor1_cpu, tensor2_cpu, dim=1)

#     # Compute L2 distance using torch.cdist
#     l2_distance = torch.cdist(tensor1_cpu.unsqueeze(0), tensor2_cpu.unsqueeze(0), p=2).squeeze(0)

#     # Print results
#     print("Cosine Similarity:")
#     print(cosine_similarity)
#     print("\nL2-norm Distance size")
#     print(l2_distance.shape)

#     # If you only need the distance between corresponding rows
#     diagonal_l2_distance = l2_distance.diag()
#     print("L2-norm Distance between corresponding rows:")
#     print(diagonal_l2_distance)

#     # Compute L1 distance
#     # Compute the L1-norm distance
#     l1_distance = torch.cdist(tensor1_cpu.unsqueeze(0), tensor2_cpu.unsqueeze(0), p=1).squeeze(0)
#     # torch.abs(tensor1 - tensor2).sum()
#     print("\nL1-norm distance size:\n", l1_distance.shape)
#     print("L1-norm Distance between corresponding rows:")
#     print(l1_distance.diag())

In [12]:
# # print the tokens twice. 
# # once according to the effective one_hot then according to the effective_embedding
# print(effective_adv_one_hot.shape)
# inputs_token_ids = one_hot_inputs.argmax(dim=1)
# adv_token_ids =  effective_adv_one_hot.argmax(dim=1)
# input_ids = torch.hstack([inputs_token_ids, adv_token_ids])
# for token_id in input_ids:
#     print(f"({tokenizer.decode(token_id)})", end=' ')
#     # print(f"({tokenizer.decode(token_id)}, {token_id.item()})", end=' ')
# print('\n')

# final_prompt_embeds = torch.hstack([embeddings_user, effective_adv_embeddings]).squeeze(0)
# for i in range(final_prompt_embeds.shape[0]):
#     if i < final_prompt_embeds.shape[0]:
#         similarities = torch.nn.functional.cosine_similarity(embed_weights, final_prompt_embeds[i])
#         token_id = similarities.argmax().cpu()
#         print(f"({tokenizer.decode(token_id)})", end=' ')
#         # print(f"({tokenizer.decode(token_id)}, {token_id.item()})", end=' ')  
# print('\n')

# # for i in range(final_prompt_embeds.shape[0]):
# #     if i < final_prompt_embeds.shape[0]:
# #         similarities = torch.nn.functional.cosine_similarity(embed_weights, final_prompt_embeds[i])
# #         token_id = similarities.argmax().cpu()
# #         print(f"({token_id}, {similarities[token_id].cpu().item(): .1f})",end=' ')  


In [13]:
# # Use the effective_adv_embeddings & effective_adv_one_hot to test the Attack
# # print("\n###### Customized generate() function; Using Adversarial Embeddings ########\n")
# # final_prompt_embeds = torch.hstack([embeddings_user, effective_adv_embeddings])
# # generated_tokens = generate(model, final_prompt_embeds, num_tokens)
# # generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
# # print("OUTPUT\n", generated_text)

# # print("\n###### Model.generate() function; Using Adversarial Embeddings ########\n")
# # # Use the Last embeddings_adv
# # final_prompt_embeds = torch.hstack([embeddings_user, embeddings_adv])
# # generated_text = model.generate(inputs_embeds=final_prompt_embeds, max_length=num_tokens).squeeze()
# # print(tokenizer.decode(generated_text, skip_special_tokens=True))

# """
# print("\n###### Model.generate() function; Using Adversarial Embeddings ########\n")
# final_prompt_embeds = torch.hstack([embeddings_user, effective_adv_embeddings])
# for i in range(10):
#     print(f"________________________{i+1}________________________")
#     generated_text = model.generate(inputs_embeds=final_prompt_embeds, max_length=num_tokens).squeeze()
#     print(tokenizer.decode(generated_text, skip_special_tokens=True))
# """
# inputs_token_ids = one_hot_inputs.argmax(dim=1)
# adv_token_ids =  effective_adv_one_hot.argmax(dim=1)
# input_ids = torch.hstack([inputs_token_ids, adv_token_ids])
# print("\n\n\n###### Model.generate() function; Using Adversarial Token IDs ########\n")
# for i in range(10):
#     print(f"________________________{i+1}________________________")
#     generated_text = model.generate(input_ids.unsqueeze(0), max_length=200)[0]
#     print(tokenizer.decode(generated_text, skip_special_tokens=True))

In [14]:
# Example usage:
# s: Updated token (e.g., logits or any real-valued tensor)
# s = torch.tensor([0.2, 0.1, -0.1, 0.4, 0.3])
# projected_s = simplex_projection(s)
# print(projected_s)

# def simplex_projection(s):
#     s = s.flatten()
#     mu, _ = torch.sort(s, descending=True)
#     cumulative_sum = torch.cumsum(mu, dim=0) - 1
#     rho = torch.sum((mu - cumulative_sum / (torch.arange(1, mu.shape[0] + 1, device=mu.device))) > 0)
#     psi = cumulative_sum[rho - 1] / rho
#     p = torch.clamp(s - psi, min=0)
#     return p

In [15]:
# Example usage:
# s = torch.tensor([0.4, 0.3, 0.2, 0.1, 0.0])
# projected_s = simplex_projection(s)
# print('simplex', projected_s)
# # s: Relaxed token (e.g., probabilities)
# projected_s = project_to_entropy(s)
# print('entropy',projected_s)


# def entropy_projection(p, target_entropy=2):
#   """
#   Projects a vector onto the simplex with a target entropy.

#   Args:
#       p: A PyTorch tensor of shape (|T|,) representing the relaxed token probabilities.
#       target_entropy: A float representing the desired entropy of the output distribution.

#   Returns:
#       A PyTorch tensor of shape (|T|,) representing the entropy-regularized probabilities.
#   """
#   # Check if projection is necessary (skip if already within target entropy)
#   mask = torch.gt(p, 0)  # Indicator for non-zero elements
#   center = torch.div(mask.float(), torch.sum(mask, dim=0))  # Average of non-zero elements
#   radius = torch.sqrt(1.0 / (target_entropy - 1.0) * torch.sum(mask, dim=0))  # Radius based on target entropy
#   norm = torch.linalg.norm(p - center)  # Norm of the difference
#   skip_projection = torch.gt(radius, norm)  # Skip if norm is less than radius

#   # Apply simplex projection if necessary
#   projected_p = torch.where(skip_projection, p, torch.nn.functional.relu(radius / norm * (p - center) + center))
#   output = simplex_projection(projected_p)
#   # Ensure the output is the same dtype as the input
#   return output.to(p.dtype)